In [ ]:
## Example from BigQuery UI
"""
CREATE MODEL `sp-msds-434-dl-sec55.iris_bq.iris_mlog_model`
OPTIONS (model_type="logistic_reg", max_iterations=10, input_label_cols=["species"])
AS SELECT * FROM `sp-msds-434-dl-sec55.iris_bq.training_data`

SELECT * FROM 
ML.PREDICT(MODEL `sp-msds-434-dl-sec55.iris_bq.iris_mlog_model`, 
  ( SELECT sepal_length, sepal_witdh, petal_length, petal_width FROM `sp-msds-434-dl-sec55.iris_bq.training_data` ) )
"""

## Example from python
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
import yfinance as yf
import sys
import datetime
import logging
import matplotlib.pyplot as plt
import json
curr = yf.Ticker("EURUSD=X")
# This will be a configurable call in custom function
df = curr.history(period="1y", interval="1h")
data = df[["Open", "High", "Low", "Close"]]

idx_tr = np.random.choice(np.arange(data.shape[0]), (np.round(data.shape[0]*0.75).astype(int)), False)
idx_te = set(np.arange(data.shape[0])) - set(idx_tr)
data["validation"] = "train"
data.loc[list(idx_te), "validation"] = "test"

client = bigquery.Client()

["Open", "High", "Low", "Close"]
schema = [
    bigquery.SchemaField("Open", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("High", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("Low", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("Close", "INTEGER", mode="REQUIRED")
]

table = bigquery.Table('msds-434-flask-fx-rl-2.fx_reg_bq.one_year_curr_data', schema=schema)
table = client.create_table(table)  # Make an API request.

rows_to_insert = data.values.tolist()
errors = client.insert_rows(table, rows_to_insert)

## Train the model
query_model = '''
CREATE OR REPLACE MODEL `msds-434-flask-fx-rl-2.fx_reg_bq.regression_model`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['Close']
  ) AS
SELECT * EXCEPT (validation) FROM `msds-434-flask-fx-rl-2.fx_reg_bq.one_year_curr_data` WHERE validation = 'train'
'''
model_job = client.query(query_model)

## Test set
query_pred = '''
SELECT * FROM 
ML.PREDICT(MODEL `msds-434-flask-fx-rl-2.fx_reg_bq.regression_model`, 
  ( SELECT * EXCEPT (validation) FROM `msds-434-flask-fx-rl-2.fx_reg_bq.one_year_curr_data` WHERE validation = 'test' ) )
'''
pred_job = client.query(query_pred)
out = pred_job.result().to_dataframe()



In [19]:
import yfinance as yf
import sys
import datetime
import logging
import matplotlib.pyplot as plt
import json
curr = yf.Ticker("EURUSD=X")
# This will be a configurable call in custom function
df = curr.history(period="1y", interval="1h")
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-05-16,1.1210,1.1212,1.1208,1.1210,0,0,0
2019-05-16,1.1211,1.1213,1.1208,1.1210,0,0,0
2019-05-16,1.1210,1.1213,1.1210,1.1212,0,0,0
2019-05-16,1.1211,1.1215,1.1210,1.1211,0,0,0
2019-05-16,1.1211,1.1215,1.1211,1.1215,0,0,0


In [21]:
curr = yf.Ticker("EURUSD=X")
# This will be a configurable call in custom function
df = curr.history(period="1y", interval="1h")
df.drop(columns=["Volume", "Dividends", "Stock Splits"], inplace=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-05-16,1.1210,1.1212,1.1208,1.1210,0,0,0
2019-05-16,1.1211,1.1213,1.1208,1.1210,0,0,0
2019-05-16,1.1210,1.1213,1.1210,1.1212,0,0,0
2019-05-16,1.1211,1.1215,1.1210,1.1211,0,0,0
2019-05-16,1.1211,1.1215,1.1211,1.1215,0,0,0


In [16]:
df.reset_index(inplace=True)